# func Development
## Final Project - Team 5
### PHYS-352 Winter 2021
## Orion Forowycz

# Initial Setup

In [124]:
import ctypes
from ctypes import *
from numpy.ctypeslib import ndpointer
import numpy as np
import matplotlib.pyplot as plt
import numba
import math

In [125]:
#load our C library, it's as simple as that!
lib = ctypes.CDLL("libode.so")
#rename C-based solve_ode() function into solve_ode_c()
solve_ode_c = lib.solve_ode
#in order to call a C function, we need to define:
# * the return data type
solve_ode_c.restype = None
# * function argument types
solve_ode_c.argtypes = [
    ndpointer(ctypes.c_double), 
    ndpointer(ctypes.c_double),
    ctypes.c_double,
    ctypes.c_int,
    ctypes.c_int,
    ctypes.c_int,
    ndpointer(ctypes.c_double),
    ctypes.CFUNCTYPE(None,c_double, POINTER(c_double), POINTER(c_double), POINTER(c_double))
]

#In order to "hide" from the end user the C "guts" of the library, 
#let's create a python "wrapper function" for our ODE solver
def solve_ode(fun, t_span, nsteps, y0, method = "RK4", args = None ):
    """
    Takes in the right-hand side function fun, the time range t_span, 
    the number of time steps nsteps, and the initial condition vector y0.
    
    Keyword arguments: 
    method -- one of "Euler", "Euler-Cromer", "RK2", "RK4" ODE solution methods
    args   -- arguments to pass to the right-hand side function fun()
    
    Returns: the pair t,y of time and solution vector.
    """
    t_span = np.asarray(t_span,dtype=np.double)
    t = np.linspace(t_span[0],t_span[1],nsteps+1,dtype=np.double)
    nvar = len(y0)
    y = np.zeros([nsteps+1,nvar],dtype=np.double,order='C')
    #assign initial conditions
    y0 = np.asarray(y0,dtype=np.double)
    y[0,:] = y0
    #check if the supplied function is numba-based CFunc
    if("ctypes" in dir(fun)):
        #numba-based, we can use it right away
        fun_c = fun.ctypes
    else:
        #otherwise, we need to wrap the python function into CFUNCTYPE
        FUNCTYPE = CFUNCTYPE(None,c_double, POINTER(c_double), POINTER(c_double), POINTER(c_double))
        #create a C-compatible function pointer
        fun_c = FUNCTYPE(fun)
    #compute preliminaries to call the C function library
    dt = (t_span[1]-t_span[0])/nsteps
    if args is not None: args = np.asarray(args,dtype=np.double)
    if method in ["RK2", "RKO2"]:
        order = 2
    elif method in ["Euler"]:
        order = 1
    elif method in ["Euler-Cromer"]:
        order = -1
    else:
        #default
        order = 4
    
    #make a call to the C library function
    solve_ode_c(t,y,dt,nsteps,nvar,order,args,fun_c)

    return t,y

# func Setup

time unit: year

length unit: AU

In [ ]:
from numba import cfunc, types

c_sig = types.void(types.double,
                   types.CPointer(types.double),
                   types.CPointer(types.double),
                   types.CPointer(types.double))

@cfunc(c_sig)
def func(t,a,params,dydt):
    """
    Takes the time and solution vector y, as well as parameters in the params array,
    and populates the dydt array with the values of the right-hand side function 
    f(t,y).
    Returns: nothing
    """

    #Particle Input
    x = a[0]
    v_x = a[1]
    y = a[2]
    v_y = a[3]
    GM_S = params[0]
    r = np.sqrt(x**2 + y**2)

    #Jupiter
    GM_J = params[1]
    r_J = params[2]
    period_J = params[3]
    theta_J = 2*np.pi*t/period_J
    x_J = r_J*np.cos(theta_J)
    y_J = r_J*np.sin(theta_J)
    x_diff = x - x_J
    y_diff = y - y_J
    r_diff = np.sqrt(x_diff**2 + y_diff**2)
    
    #Particle Output
    dydt[0] = v_x #dx/dt
    dydt[1] = -GM_S/(r**3)*x - GM_J/(r_diff**3)*x_diff #dv_x/dt
    dydt[2] = v_y #dy/dt
    dydt[3] = -GM_S/(r**3)*y - GM_J/(r_diff**3)*y_diff #dv_y/dt